In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Prepare Data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import trefide.preprocess as preprocess
import scipy.io as io

In [4]:
# Load Data
stim = io.loadmat("../data/sampleMovie.mat")['stimulationSignal'][1:-50].squeeze()
data = io.loadmat("../data/post_demix.mat")
N, T = data['C'].shape
signals = data['C']

In [ ]:
signals, trends, stim, disc_idx = preprocess.detrend(signals, stim, disc_idx=np.array([5]))

In [ ]:
signals = signals.copy() # make sure signals contiguous in memory

In [ ]:
T = len(stim)
idx = np.random.randint(0, N)
signal = signals[idx,:]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(np.arange(T), signal+trends[idx,:], 'r')
ax.plot(np.arange(T), trends[idx,:], 'b')

ax.set(xlabel='time', ylabel='Fluorescence',
       title='Slow Trend Fluorescence Neuron {}'.format(idx+1))
ax.grid()

plt.show()

# Instantial and Fit A Trend Filtering Object for Each Signal

In [1]:
from trefide.temporal import TrendFilter

ImportError: /home/ian/devel/trefide/trefide/utils.cpython-36m-x86_64-linux-gnu.so: undefined symbol: _Z5welchmiimPKdPdPv

In [ ]:
filters = [TrendFilter(len(signal)) for signal in signals]

In [ ]:
denoised = np.asarray([filt.denoise(signal) for signal, filt in zip(signals, filters)])

In [ ]:
fig, ax = plt.subplots(nrows=int(np.ceil(len(signals)/2)), ncols=2, figsize=(16, 3 * np.ceil(len(signals)/2)))

for idx, (noisey, smooth) in enumerate(zip(signals, denoised)):
    ax[int(np.floor(idx/2)), int(idx%2)].plot(np.arange(T), noisey + trends[idx], 'r')
    ax[int(np.floor(idx/2)), int(idx%2)].plot(np.arange(T), smooth + trends[idx,:], 'b')
    ax[int(np.floor(idx/2)), int(idx%2)].set(xlabel='time', ylabel='Fluorescence',
                                             title='Slow Trend Fluorescence Neuron {}'.format(idx+1))
    ax[int(np.floor(idx/2)), int(idx%2)].grid()

plt.tight_layout()
plt.show()